In [1]:
from confluent_kafka import Consumer, KafkaError
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.protobuf import ProtobufDeserializer
from confluent_kafka.serialization import SerializationContext, MessageField
import my_event_pb2

In [2]:
def calculate_square(value):
    return value * value

In [3]:
schema_registry_conf = {'url': 'http://schema-registry:8081'}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

deserializer_conf = {'use.deprecated.format': False}
protobuf_deserializer = ProtobufDeserializer(my_event_pb2.MyEvent, deserializer_conf)

/opt/conda/lib/python3.11/site-packages/confluent_kafka/schema_registry/protobuf.py:513: UserWarning: MessageFactory class is deprecated. Please use GetMessageClass() instead of MessageFactory.GetPrototype. MessageFactory class will be removed after 2024.
  self._msg_class = MessageFactory().GetPrototype(descriptor)


In [4]:
c = Consumer({
    'bootstrap.servers': 'kafka-1:29092,kafka-2:29093,kafka-3:29094',
    'group.id': 'mygroup',
    'auto.offset.reset': 'earliest'
})

c.subscribe(['my-topic'])

In [5]:
while True:
    msg = c.poll(5.0)

    if msg is None:
        continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
            continue
        else:
            print(msg.error())
            break

    event = protobuf_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))
    
    square = event.value * event.value
    print(f'Received event with timestamp: {event.timestamp.seconds}')
    print(f'Square Calculator Consumer - Original: {event.value}, Square: {square}\n')

c.close()

Received event with timestamp: 1725445691
Square Calculator Consumer - Original: 56, Square: 3136

Received event with timestamp: 1725445701
Square Calculator Consumer - Original: 51, Square: 2601

Received event with timestamp: 1725445716
Square Calculator Consumer - Original: 74, Square: 5476

Received event with timestamp: 1725445706
Square Calculator Consumer - Original: 46, Square: 2116

Received event with timestamp: 1725445711
Square Calculator Consumer - Original: 64, Square: 4096

Received event with timestamp: 1725445721
Square Calculator Consumer - Original: 75, Square: 5625

Received event with timestamp: 1725445746
Square Calculator Consumer - Original: 92, Square: 8464

Received event with timestamp: 1725445751
Square Calculator Consumer - Original: 65, Square: 4225

Received event with timestamp: 1725445756
Square Calculator Consumer - Original: 60, Square: 3600

Received event with timestamp: 1725445761
Square Calculator Consumer - Original: 78, Square: 6084

Received e

KeyboardInterrupt: 